In [2]:
import numpy as np
import pandas as pd
import random
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import fbeta_score
import os
import pickle
from tqdm import tqdm_notebook
from time import ctime

In [3]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import models

In [4]:
DATA_DIR = '../data/'

In [5]:
def read_image(path):
    im = cv2.imread(path)
    return cv2.cvtColor(im, cv2.COLOR_BGR2RGB)

In [6]:
def normalize(im):
    """Normalizes images with Imagenet stats."""
    imagenet_stats = np.array([[0.485, 0.456, 0.406], [0.229, 0.224, 0.225]])
    return (im - imagenet_stats[0]) / imagenet_stats[1]

In [7]:
# data partitions
with open(os.path.join(DATA_DIR, 'partition.p'), 'rb') as f:
    partition = pickle.load(f)

## Data Augmentation

In [8]:
def crop(im, r, c, target_r, target_c):
    return im[r:r+target_r, c:c+target_c]

# random crop to the original size
def random_crop(x, r_pix=8):
    """Returns a random crop"""
    r, c, *_ = x.shape
    r, c, *_ = x.shape
    c_pix = round(r_pix*c/r)
    rand_r = random.uniform(0, 1)
    rand_c = random.uniform(0, 1)
    start_r = np.floor(2*rand_r*r_pix).astype(int)
    start_c = np.floor(2*rand_c*c_pix).astype(int)
    return crop(x, start_r, start_c, r-2*r_pix, c-2*c_pix)

def center_crop(x, r_pix=8):
    r, c, *_ = x.shape
    c_pix = round(r_pix*c/r)
    return crop(x, r_pix, c_pix, r-2*r_pix, c-2*c_pix)


def rotate_cv(im, deg, mode=cv2.BORDER_REFLECT, interpolation=cv2.INTER_AREA):
    """ Rotates an image by deg degrees"""
    r, c, *_ = im.shape
    M = cv2.getRotationMatrix2D((c/2, r/2), deg, 1)
    return cv2.warpAffine(im, M, (c,r), borderMode=mode, 
                          flags=cv2.WARP_FILL_OUTLIERS+interpolation)

## Dataset

In [9]:
class PlanetDataset(Dataset):
    def __init__(self, img_folder, list_IDs, csv_path, transforms=False):
        self.list_IDs = list_IDs
        self.df = pd.read_csv(csv_path)
        self.img_folder = img_folder
        self.mlb = MultiLabelBinarizer()
        self.y = self.mlb.fit_transform([tag.split() for tag in self.df.tags])
        self.y = dict(zip(self.df['image_name'], self.y))
        self.transforms = transforms

    def __len__(self):
        return len(self.list_IDs)

    def __getitem__(self, idx):
        name = self.list_IDs[idx]
        img_path = os.path.join(self.img_folder, name + '.jpg')
        x = cv2.imread(img_path).astype(np.float32)
        x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB) / 255
        if self.transforms:
            rdeg = (np.random.random() - 0.50) * 20
            x = rotate_cv(x, rdeg)
            x = random_crop(x)
            if np.random.random() > 0.5:
                x = np.fliplr(x).copy()
        else:
            x = center_crop(x)
        x = normalize(x)
        return np.rollaxis(x, 2), self.y[name]

## Optimizer

In [10]:
def lr_scheduler(epoch, factor, init_lr=0.01, lr_decay_epoch=7):
    '''
    Decay learning rate by a factor every lr_decay_epoch epochs.
    '''
    lr = init_lr * (factor**(epoch // lr_decay_epoch))
    if epoch % lr_decay_epoch == 0:
        print("Setting base LR to %.8f" % lr)
    return lr

In [11]:
def create_optimizer(model, optimizer, lr0, diff_lr_factors):
    '''
    Creates an optimizer for a NN segmented into groups, with a differential
    learning rate across groups according to a multiplicative factor for each
    group given by group_lrs
    '''
    n_groups = len(diff_lr_factors)
    param_groups = [list(model.groups[i].parameters()) for i in range(n_groups)]
    params = [{'params': p, 'lr': lr0/diff_factor} for p, diff_factor in zip(param_groups, diff_lr_factors)]
    return optimizer(params)

## Validation

In [12]:
def validate(model, val_dl, threshold):
    model.eval()
    total_loss = 0
    true_labels, predictions = [], []
    with torch.no_grad():
        for data, target in tqdm_notebook(val_dl, desc='Validation metrics',
                                 total=len(val_dl)):
            true_labels.append(target.cpu().numpy())
            data, target = data.cuda().float(), target.cuda().float()

            pred = model(data)
            predictions.append(F.sigmoid(pred).cpu().numpy())
            total_loss += F.binary_cross_entropy_with_logits(pred,
                                                             target).item()

        avg_loss = total_loss / len(val_dl)
        predictions = np.vstack(predictions)
        true_labels = np.vstack(true_labels)
        f2_score = fbeta_score(true_labels, predictions > threshold,
                               beta=2, average='samples')
        return f2_score, avg_loss

## Models

In [31]:
vgg = models.vgg19(pretrained=True)

In [13]:
vgg = models.vgg16(pretrained=True)

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /home/ubuntu/.cache/torch/checkpoints/vgg16-397923af.pth
100%|██████████| 553433881/553433881 [00:10<00:00, 54869693.15it/s]


In [37]:
class VGG19(nn.Module):
    def __init__(self, num_classes):
        super(VGG19, self).__init__()
        self.vgg = models.vgg19(pretrained=True)
        # freezing parameters
        for param in self.vgg.parameters():
            param.requires_grad = False
        # modify the final linear layer
        num_features = self.vgg.classifier[6].in_features
        self.vgg.classifier = self.vgg.classifier[:6]
        # separate layers into two groups
        layers = list(self.vgg.children())
        self.groups = nn.ModuleList([nn.Sequential(*h) for h in [layers[:2], layers[2:]]])
        self.groups.append(nn.Linear(num_features, num_classes))

    def forward(self, x):
        x = self.groups[0](x)
        x = x.view(x.size(0), -1)
        for group in self.groups[1:]:
            x = group(x)
        return x

    def unfreeze(self,  group_idx: int):
        group = self.groups[group_idx]
        parameters = filter(lambda x: hasattr(x, 'requires_grad'),
                            group.parameters())
        for p in parameters:
            p.requires_grad = True

In [38]:
# model
model = VGG19(num_classes=17).cuda()

# datasets
train_ds = PlanetDataset(os.path.join(DATA_DIR, 'train-jpg'), 
                        partition['inner_train'],
                        os.path.join(DATA_DIR, 'train_v2.csv'),
                        True)

val_ds = PlanetDataset(os.path.join(DATA_DIR, 'train-jpg'),
                    partition['validation'],
                    os.path.join(DATA_DIR, 'train_v2.csv'))

# data loaders
batch_size = 64
train_dl = DataLoader(train_ds,
                    batch_size=batch_size,
                    num_workers=4,
                    pin_memory=True,
                    shuffle=True)

val_dl = DataLoader(val_ds,
                    batch_size=batch_size,
                    num_workers=4,
                    pin_memory=True)

Testing data flow through the model

In [18]:
data, label = iter(train_dl).next()
data = data.cuda().float()
label = label.cuda().float()

In [14]:
features = model.vgg.features(data)
avgpool = model.vgg.avgpool(features)
avgpool = avgpool.view(avgpool.size(0), -1)
classifier = model.vgg.classifier(avgpool)

In [19]:
model.eval()
with torch.no_grad():
    y_pred = model(data)

Testing training loop with constant learning rate

In [27]:
learning_rate = 0.01
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# training loop
epochs = 10
for epoch in range(epochs):
    print(ctime())
    iterations = epochs*len(train_dl)
    idx = 0
    total_loss = 0
    # training loop
    for batch_idx, (data, target) in enumerate(train_dl):
        data, target = data.cuda().float(), target.cuda().float()
        output = model(data)
        loss = F.binary_cross_entropy_with_logits(output, target)
        total_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        idx += 1
        if batch_idx % 100 == 0:
            print("Epoch %d (Batch %d / %d)\t Train loss: %.3f" % \
                (epoch+1, batch_idx, len(train_dl), loss.item()))
    # train loss
    train_loss = total_loss / len(train_dl)
    print("Epoch %d\t Train loss: %.3f" % (epoch+1, train_loss))
    val_f2_score, val_loss = validate(model, val_dl, 0.2)
    print("Epoch %d \t Validation loss: %.3f, F2 score: %.3f" % (epoch+1, val_loss, val_f2_score))

Sat Aug  3 01:00:01 2019
Epoch 1 (Batch 0 / 506)	 Train loss: 0.188
Epoch 1 (Batch 100 / 506)	 Train loss: 0.245
Epoch 1 (Batch 200 / 506)	 Train loss: 0.217
Epoch 1 (Batch 300 / 506)	 Train loss: 0.149
Epoch 1 (Batch 400 / 506)	 Train loss: 0.196
Epoch 1 (Batch 500 / 506)	 Train loss: 0.200
Epoch 1	 Train loss: 0.210


/home/ubuntu/anaconda3/envs/planet-imagery/lib/python3.7/site-packages/torch/nn/functional.py:1386: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")



Epoch 1 	 Validation loss: 0.175, F2 score: 0.859
Sat Aug  3 01:06:19 2019


/home/ubuntu/anaconda3/envs/planet-imagery/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)


Epoch 2 (Batch 0 / 506)	 Train loss: 0.171


Traceback (most recent call last):
  File "/home/ubuntu/anaconda3/envs/planet-imagery/lib/python3.7/multiprocessing/queues.py", line 242, in _feed
    send_bytes(obj)
  File "/home/ubuntu/anaconda3/envs/planet-imagery/lib/python3.7/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/home/ubuntu/anaconda3/envs/planet-imagery/lib/python3.7/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/home/ubuntu/anaconda3/envs/planet-imagery/lib/python3.7/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe
Traceback (most recent call last):
  File "/home/ubuntu/anaconda3/envs/planet-imagery/lib/python3.7/multiprocessing/queues.py", line 242, in _feed
    send_bytes(obj)
  File "/home/ubuntu/anaconda3/envs/planet-imagery/lib/python3.7/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset +

KeyboardInterrupt: 

Testing training loop with differential and decaying learning rate

In [43]:
# model
MODEL = VGG19(num_classes=17).cuda()

# datasets
train_ds = PlanetDataset(os.path.join(DATA_DIR, 'train-jpg'), 
                        partition['inner_train'],
                        os.path.join(DATA_DIR, 'train_v2.csv'),
                        True)

val_ds = PlanetDataset(os.path.join(DATA_DIR, 'train-jpg'),
                    partition['validation'],
                    os.path.join(DATA_DIR, 'train_v2.csv'))

# data loaders
batch_size = 64
train_dl = DataLoader(train_ds,
                    batch_size=batch_size,
                    num_workers=4,
                    pin_memory=True,
                    shuffle=True)

val_dl = DataLoader(val_ds,
                    batch_size=batch_size,
                    num_workers=4,
                    pin_memory=True)

# optimizer
BASE_OPTIMIZER = optim.Adam
DIFF_LR_FACTORS = [9, 3, 1]
INIT_LR_0 = 0.01
EPOCHS = 40

best_score = 0.0
# create optimizer with differential learning rates
optimizer = create_optimizer(model, BASE_OPTIMIZER, INIT_LR_0, DIFF_LR_FACTORS)
iterations = EPOCHS*len(train_dl)
idx = 0
for epoch in range(EPOCHS):
    lr0 = lr_scheduler(epoch, 0.1, INIT_LR_0, 5)  # set base lr for this epoch
    optimizer = create_optimizer(model, BASE_OPTIMIZER, lr0, DIFF_LR_FACTORS)
    total_loss = 0
    # training loop
    for batch_idx, (data, target) in enumerate(train_dl):
        data, target = data.cuda().float(), target.cuda().float()
        output = model(data)
        loss = F.binary_cross_entropy_with_logits(output, target)
        total_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        idx += 1
        # unfreeze deeper layers sequentially
        if idx == int(0.1*iterations):
            model.unfreeze(1)
            print("Iteration %d: Unfreezing group 1" % idx)
        if idx == int(0.2*iterations):
            model.unfreeze(0)
            print("Iteration %d: Unfreezing group 0" % idx)
        if batch_idx % 100 == 0:
            print("Epoch %d (Batch %d / %d)\t Train loss: %.3f" % \
                (epoch+1, batch_idx, len(train_dl), loss.item()))
    # train loss
    train_loss = total_loss / len(train_dl)
    print("Epoch %d\t Train loss: %.3f" % (epoch+1, train_loss))
    # validation scores
    val_f2_score, val_loss = validate(model, val_dl, 0.2)
    print("Epoch %d \t Validation loss: %.3f, F2 score: %.3f" % \
        (epoch+1, val_loss, val_f2_score))

Setting base LR to 0.01000000
Epoch 1 (Batch 0 / 506)	 Train loss: 0.704
Epoch 1 (Batch 100 / 506)	 Train loss: 0.176
Epoch 1 (Batch 200 / 506)	 Train loss: 0.235
Epoch 1 (Batch 300 / 506)	 Train loss: 0.192
Epoch 1 (Batch 400 / 506)	 Train loss: 0.167
Epoch 1 (Batch 500 / 506)	 Train loss: 0.249
Epoch 1	 Train loss: 0.192



Epoch 1 	 Validation loss: 0.159, F2 score: 0.864
Epoch 2 (Batch 0 / 506)	 Train loss: 0.149


Traceback (most recent call last):
  File "/home/ubuntu/anaconda3/envs/planet-imagery/lib/python3.7/multiprocessing/queues.py", line 242, in _feed
    send_bytes(obj)
  File "/home/ubuntu/anaconda3/envs/planet-imagery/lib/python3.7/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/home/ubuntu/anaconda3/envs/planet-imagery/lib/python3.7/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/home/ubuntu/anaconda3/envs/planet-imagery/lib/python3.7/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe
Traceback (most recent call last):
  File "/home/ubuntu/anaconda3/envs/planet-imagery/lib/python3.7/multiprocessing/queues.py", line 242, in _feed
    send_bytes(obj)
  File "/home/ubuntu/anaconda3/envs/planet-imagery/lib/python3.7/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset +

KeyboardInterrupt: 